<font color='red'>使用前请创建相应的文件夹或修改文件路径，为防止提交相同的结果被官方封号，至少请修改随机数种子以后再训练提交！！！</font>

目录结构：
```
.
├── dataset # 原始数据集
│   ├── entprise_evaluate.csv
│   ├── entprise_submit.csv
│   ├── train
│   │   ├── annual_report_info.csv
│   │   ├── base_info.csv
│   │   ├── change_info.csv
│   │   ├── entprise_info.csv
│   │   ├── news_info.csv
│   │   ├── other_info.csv
│   │   └── tax_info.csv
│   └── train.zip
├── feature # 特征工程中间文件
│   ├── cat_cols.json 
│   ├── test.csv 
│   └── train.csv 
├── importance.csv 
├── result # 提交结果
│   └── cat_sub.csv 
└── siriyang_catboost_baseline.ipynb
```

# 特征提取

## 读取原始数据集和导包

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json

base_info = pd.read_csv("./dataset/train/base_info.csv")
annual_report_info = pd.read_csv("./dataset/train/annual_report_info.csv")
tax_info = pd.read_csv("./dataset/train/tax_info.csv")
change_info = pd.read_csv("./dataset/train/change_info.csv")
news_info = pd.read_csv("./dataset/train/news_info.csv")
other_info = pd.read_csv("./dataset/train/other_info.csv")
train_label = pd.read_csv("./dataset/train/entprise_info.csv")
test = pd.read_csv("./dataset/entprise_evaluate.csv")

## 特征工程

In [2]:
def get_feature():
    
    cat_cols=[
    'base_oplocdistrict',
    'base_industryco',
    'base_enttype',
    'base_enttypeitem',
    'base_state',
    'base_orgid',
    'base_jobid',
    'base_adbusign',
    'base_townsign',
    'base_regtype',
    'base_compform',
    'base_venind',
    'base_enttypeminu',
    'base_protype',
    'base_enttypegb',
    ]
    
    # -------------------------提取base_info表特征---------------------------
    
    col=['oplocdistrict','industryco','enttype','enttypeitem',
        'state','orgid','jobid','adbusign','townsign','regtype','empnum',
        'compform','parnum','exenum','venind','enttypeminu','protype',
        'regcap','reccap','forreccap','forregcap','congro','enttypegb']
    feature=base_info[['id']+col]
    feature.rename(columns={i:'base_'+i for i in col },inplace=True)
    
    # 将industryphy字段进行编码
    id_to_base_industryphy=list(set(base_info['industryphy']))
    base_industryphy_to_id={k:v+1 for v,k in enumerate(id_to_base_industryphy)}
    feature['base_industryphy_id']=base_info['industryphy'].map(lambda x:base_industryphy_to_id[x])
    cat_cols.append('base_industryphy_id')
    
    # -------------------------提取annual_report_info表特征---------------------------
    
    
    # -------------------------提取tax_info表特征---------------------------

    
    # -------------------------提取news_info表特征---------------------------
    
    
    # -------------------------提取other_info表特征---------------------------
    
    
    display(feature)
    
    for i in cat_cols:
        print(i)
        
    # 将分类特征的字段名保存，以便后期训练时读取
    with open('./feature/cat_cols.json', 'w') as fw:
        # 将字典转化为字符串
        # json_str = json.dumps(data)
        # fw.write(json_str)
        # 上面两句等同于下面这句
        json.dump(cat_cols,fw)
    
    return feature

In [3]:

if __name__ == "__main__":
    from datetime import datetime
    start = datetime.now()
    print(start.strftime('%Y-%m-%d %H:%M:%S'))
    
    
    feature=get_feature()
    
    train_feature = train_label.merge(feature, how='left', on='id', copy=False)
    test_feature = test.merge(feature, how='left', on='id', copy=False)
    
    display(train_feature)
    display(test_feature)
    for i in feature.columns:
        print(i)
    
    train_feature.to_csv('./feature/train.csv',index=False)
    test_feature.to_csv('./feature/test.csv',index=False)
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print('time costed is: %d s' % (int((datetime.now() - start).seconds)))


2020-11-01 11:15:37


/home/ubuntu/anaconda3/envs/bigdata/lib/python3.6/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ubuntu/anaconda3/envs/bigdata/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,base_oplocdistrict,base_industryco,base_enttype,base_enttypeitem,base_state,base_orgid,base_jobid,base_adbusign,base_townsign,...,base_venind,base_enttypeminu,base_protype,base_regcap,base_reccap,base_forreccap,base_forregcap,base_congro,base_enttypegb,base_industryphy_id
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,7513.0,1100,1150.0,6,340223010010000000,340200000000115392,0,0,...,NaN,1151.0,NaN,50.0,NaN,NaN,NaN,NaN,1151,4
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,8090.0,9600,NaN,6,340222060010000000,340200000000112114,0,1,...,3.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,9600,20
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,9053.0,1100,1150.0,6,340202010010000000,400000000000753910,0,0,...,NaN,1151.0,NaN,100.0,NaN,NaN,NaN,NaN,1151,6
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,7212.0,4500,4540.0,6,340221010010000000,400000000000013538,0,1,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,4540,5
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,8810.0,1100,1130.0,7,340200000000000000,400000000000283237,0,0,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,1130,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24860,f1c1045b13d18329a2bd99d2a7e2227688c0d69bf1d1e325,340225,8131.0,9600,NaN,6,340200000000116780,341400000000011622,0,1,...,3.0,NaN,NaN,20.0,NaN,NaN,NaN,NaN,9600,20
24861,f000950527a6feb6bde38216d7cbbf32e66d3a3a96d4dbda,340207,6790.0,4500,4530.0,6,340200000000000000,340200000000115797,0,1,...,NaN,4533.0,NaN,110.0,0.0,NaN,NaN,NaN,4533,18
24862,da8691b210adb3f65b43370d3a362f4aa1d3b16b5ba0c9d7,340207,8111.0,9600,NaN,6,340207030010000000,340200000000115275,0,1,...,3.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,9600,20
24863,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,340225,8090.0,1100,1130.0,7,340200000000116750,341400000000015220,0,1,...,3.0,NaN,NaN,260.0,260.0,NaN,NaN,NaN,1130,20


base_oplocdistrict
base_industryco
base_enttype
base_enttypeitem
base_state
base_orgid
base_jobid
base_adbusign
base_townsign
base_regtype
base_compform
base_venind
base_enttypeminu
base_protype
base_enttypegb
base_industryphy_id


,id,label,base_oplocdistrict,base_industryco,base_enttype,base_enttypeitem,base_state,base_orgid,base_jobid,base_adbusign,...,base_venind,base_enttypeminu,base_protype,base_regcap,base_reccap,base_forreccap,base_forregcap,base_congro,base_enttypegb,base_industryphy_id
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1,340202,6760.0,4500,4530.0,6,340202010010000000,340200000020001145,0,...,NaN,4533.0,NaN,42205.0,37980.0,NaN,NaN,NaN,4533,18
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0,340207,8060.0,9600,NaN,6,340207030010000000,340200000020002162,0,...,3.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,9600,20
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0,340208,7590.0,1100,1130.0,6,340206010010000000,400000000000753916,0,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,1130,4
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0,340202,7514.0,1100,1150.0,6,340200000000000000,400000000000386531,0,...,NaN,1151.0,NaN,1285.0,NaN,NaN,NaN,NaN,1151,4
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0,340225,8111.0,9600,NaN,6,400000000000422283,400000000000735900,0,...,3.0,NaN,NaN,100.0,NaN,NaN,NaN,NaN,9600,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14860,216bd2aaf4d079242082b9d367435a08dc120b8de3e4bbb0,0,340203,7511.0,1100,1150.0,6,340203010010000000,400000000000753907,0,...,NaN,1151.0,NaN,30.0,NaN,NaN,NaN,NaN,1151,4
14861,e9f7b28ec10e0470ced7ec6501a54dd4a3f93e5e67eb12ba,0,340221,8219.0,1100,1150.0,6,340221010010000000,400000000000245172,0,...,NaN,1151.0,NaN,10.0,NaN,NaN,NaN,NaN,1151,20
14862,9c7fa510616a6830cb4645157d59aec8953184da2f5c6f45,0,340222,8040.0,9600,NaN,6,340222060010000000,340200000020003303,0,...,3.0,NaN,NaN,20.0,NaN,NaN,NaN,NaN,9600,20
14863,f000950527a6feb6648ce953b47101e15608d6422999cbe1,0,340202,8499.0,1100,1130.0,7,340200000000000000,400000000000386531,0,...,NaN,NaN,NaN,70.0,0.0,NaN,NaN,NaN,1130,8


,id,score,base_oplocdistrict,base_industryco,base_enttype,base_enttypeitem,base_state,base_orgid,base_jobid,base_adbusign,...,base_venind,base_enttypeminu,base_protype,base_regcap,base_reccap,base_forreccap,base_forregcap,base_congro,base_enttypegb,base_industryphy_id
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,NaN,340222,7461.0,1100,1130.0,6,340206010010000000,340200000000101140,0,...,NaN,NaN,NaN,1000.0,200.0,NaN,NaN,NaN,1130,4
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,NaN,340200,7212.0,1100,1130.0,6,340203010010000000,340200000000100362,0,...,NaN,NaN,NaN,500.0,500.0,0.0,0.0,0.0,1130,5
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,NaN,340221,8090.0,1100,1150.0,6,340221010010000000,340200000000112737,0,...,NaN,1151.0,NaN,100.0,NaN,NaN,NaN,NaN,1151,20
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,NaN,340202,8040.0,9600,NaN,6,340200000000112185,340200000000112948,0,...,3.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,9600,20
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,NaN,340221,8219.0,1100,1150.0,6,340221010010000000,340200000000112739,0,...,NaN,1151.0,NaN,10.0,0.0,NaN,NaN,NaN,1151,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,f000950527a6feb6b9e9c5a82689e87ee128abcf72ca7b96,NaN,340203,5219.0,1100,1150.0,7,340203010010000000,340200000000100093,0,...,NaN,1151.0,NaN,1000.0,0.0,NaN,NaN,NaN,1151,11
9996,d8071a739aa75a3bb98b032a18ae492bb8cf7ad9e0c23acd,NaN,340200,7511.0,1100,1130.0,6,340207010010000000,340200000000101462,0,...,NaN,NaN,NaN,1200.0,1200.0,NaN,NaN,NaN,1130,4
9997,f000950527a6feb63ae3783e4b82cbd8da7b3eaf43624866,NaN,340203,7499.0,6100,6110.0,6,340200000000000000,340200000020000883,0,...,NaN,NaN,99.0,50.0,NaN,NaN,68.0,118.0,6110,4
9998,d8071a739aa75a3bf8557cd0432d5c04e2241aee9f422220,NaN,340207,7320.0,1100,1130.0,7,340207010010000000,340200000020001917,0,...,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,1130,4


id
base_oplocdistrict
base_industryco
base_enttype
base_enttypeitem
base_state
base_orgid
base_jobid
base_adbusign
base_townsign
base_regtype
base_empnum
base_compform
base_parnum
base_exenum
base_venind
base_enttypeminu
base_protype
base_regcap
base_reccap
base_forreccap
base_forregcap
base_congro
base_enttypegb
base_industryphy_id
2020-11-01 11:15:38
time costed is: 0 s


# 模型训练

## 读取构建好的特征、分类字段名和导包

In [4]:
import numpy as np
import pandas as pd
import json
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

train = pd.read_csv('./feature/train.csv')
test = pd.read_csv('./feature/test.csv')
sub = pd.read_csv("./dataset/entprise_submit.csv")

train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

feat = [c for c in train.columns if c not in ['id','label']]

cat_cols=[]
with open('./feature/cat_cols.json','r') as f:
        cat_cols = json.load(f)
for i in cat_cols:
        print(i)

for i in train.columns:
    if i in cat_cols:
        train[i] = train[i].astype('str')
        test[i] = test[i].astype('str')

base_oplocdistrict
base_industryco
base_enttype
base_enttypeitem
base_state
base_orgid
base_jobid
base_adbusign
base_townsign
base_regtype
base_compform
base_venind
base_enttypeminu
base_protype
base_enttypegb
base_industryphy_id


## 训练

In [5]:

if __name__ == "__main__":
    from datetime import datetime
    start = datetime.now()
    print(start.strftime('%Y-%m-%d %H:%M:%S'))
    
    num_folds=5
    kfold = StratifiedKFold(n_splits=num_folds, random_state=1024, shuffle=False).split(train.drop(['label'],axis=1), train['label'])

    oof_probs = np.zeros(train.shape[0])
    output_probs = np.zeros((test.shape[0],5))
    offline_score = []
    feature_importance_df = pd.DataFrame()
    for fold, (train_idx, valid_idx) in enumerate(kfold):
        X_train, y_train = train[feat].iloc[train_idx], train['label'].iloc[train_idx]
        X_valid, y_valid = train[feat].iloc[valid_idx], train['label'].iloc[valid_idx]
        
        model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            task_type="CPU",
            learning_rate=0.01,
            iterations=10000,
            random_seed=2020,
            od_type="Iter",
            depth=8,
            early_stopping_rounds=500,

        )

        clf = model.fit(X_train,y_train, eval_set=(X_valid,y_valid),verbose=500,cat_features=cat_cols)
        yy_pred_valid=clf.predict(X_valid)
        y_pred_valid = clf.predict(X_valid,prediction_type='Probability')[:,-1]
        oof_probs[valid_idx] = y_pred_valid
        offline_score.append(f1_score(y_valid, yy_pred_valid))
        output_probs[:, fold] = clf.predict(test[feat],prediction_type='Probability')[:,-1]
        
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = model.feature_names_
        fold_importance_df["importance"] = model.feature_importances_
        fold_importance_df["fold"] = fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print('OOF-MEAN-F1:%.6f, OOF-STD-F1:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    feature_importance_df_ = feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False)
    # print(feature_importance_df_.head(15))
    print(feature_importance_df_)
    feature_importance_df_.to_csv("./importance.csv")
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print('time costed is: %d s' % (int((datetime.now() - start).seconds)))


2020-11-01 11:15:39


/home/ubuntu/anaconda3/envs/bigdata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0:	learn: 0.7809762	test: 0.7543424	best: 0.7543424 (0)	total: 93.9ms	remaining: 15m 39s
500:	learn: 0.8769519	test: 0.7969925	best: 0.8010076 (133)	total: 19.9s	remaining: 6m 17s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8010075567
bestIteration = 133

Shrink model to first 134 iterations.
0:	learn: 0.7801333	test: 0.7706856	best: 0.7706856 (0)	total: 37.6ms	remaining: 6m 16s
500:	learn: 0.8777429	test: 0.8474576	best: 0.8599034 (161)	total: 19.9s	remaining: 6m 16s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8599033816
bestIteration = 161

Shrink model to first 162 iterations.
0:	learn: 0.7776066	test: 0.7822581	best: 0.7822581 (0)	total: 45ms	remaining: 7m 29s
500:	learn: 0.8760125	test: 0.8456057	best: 0.8551069 (278)	total: 20s	remaining: 6m 18s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8551068884
bestIteration = 278

Shrink model to first 279 iterations.
0:	learn: 0.7777778	test: 0.7833333	best: 0.78333

## 保存结果

In [6]:
sub['score'] = np.mean(output_probs, axis=1)
print(sub['score'])
sub.to_csv("./result/cat_sub.csv")

0       0.018979
1       0.586423
2       0.004458
3       0.002345
4       0.004200
          ...   
9995    0.162103
9996    0.052757
9997    0.029583
9998    0.004465
9999    0.924557
Name: score, Length: 10000, dtype: float64
